## Heuristic Method of NLP

In [4]:
import pandas as pd
from nltk.corpus import wordnet as wn
import copy
from itertools import combinations
from nltk import word_tokenize

In [6]:
pd.read_pickle('mydf_relevant.pkl').head(10)

,id,title,url,house,text,text_len
689,343,Tom Riddle,/wiki/Tom_Riddle,Slytherin,"Voldemort was considered by many to be ""the mo...",27040
168,13,Harry Potter,/wiki/Harry_Potter,Gryffindor,"Harry was an extremely brave, loyal, and selfl...",13712
726,49,Dolores Umbridge,/wiki/Dolores_Umbridge,Slytherin,"Dolores Umbridge was an evil woman, nothing sh...",9897
703,259,Horace Slughorn,/wiki/Horace_Slughorn,Slytherin,Horace Slughorn was described as having a bumb...,8805
53,4178,Albus Dumbledore,/wiki/Albus_Dumbledore,Gryffindor,Considered to be the most powerful wizard of h...,7789
709,33,Severus Snape,/wiki/Severus_Snape,Slytherin,"A complicated, withdrawn person, Severus Snape...",7073
163,331,Peter Pettigrew,/wiki/Peter_Pettigrew,Gryffindor,Peter Pettigrew was characterised by weakness....,6602
467,53,Gilderoy Lockhart,/wiki/Gilderoy_Lockhart,Ravenclaw,Gilderoy Lockhart's defining characteristics w...,6245
229,14,Ronald Weasley,/wiki/Ronald_Weasley,Gryffindor,"Ron was a very funny person, but often emotion...",6078
646,16,Draco Malfoy,/wiki/Draco_Malfoy,Slytherin,"Draco was, in general, an arrogant, spiteful b...",5435


In [30]:
mydf_relevant.head()

,id,title,url,house,text,text_len
689,343,Tom Riddle,/wiki/Tom_Riddle,Slytherin,"Voldemort was considered by many to be ""the mo...",27040
168,13,Harry Potter,/wiki/Harry_Potter,Gryffindor,"Harry was an extremely brave, loyal, and selfl...",13712
726,49,Dolores Umbridge,/wiki/Dolores_Umbridge,Slytherin,"Dolores Umbridge was an evil woman, nothing sh...",9897
703,259,Horace Slughorn,/wiki/Horace_Slughorn,Slytherin,Horace Slughorn was described as having a bumb...,8805
53,4178,Albus Dumbledore,/wiki/Albus_Dumbledore,Gryffindor,Considered to be the most powerful wizard of h...,7789


In [32]:
'''All of these characteristics are nouns'''
trait_dict = {}
trait_dict['Gryffindor'] = ['bravery', 'nerve', 'chivalry', 'daring', 'courage']
trait_dict['Slytherin'] = ['resourcefulness', 'cunning', 'ambition', 'determination', 'self-preservation', 'fraternity',
                           'cleverness']
trait_dict['Ravenclaw'] = ['intelligence', 'wit', 'wisdom', 'creativity', 'originality', 'individuality', 'acceptance']
trait_dict['Hufflepuff'] = ['dedication', 'diligence', 'fairness', 'patience', 'kindness', 'tolerance', 'persistence',
                            'loyalty']

In [34]:
'''Scan through the "text" column in our DataFrame and count the number of times a characteristic appears'''


'Scan through the "text" column in our DataFrame and count the number of times a characteristic appears'

In [62]:
'''Show example meaning  in order to choose relavent synsets for lemmas'''
wit_synsets = wn.synsets('wit')
print (wit_synsets)
print("")
 
for wit in wit_synsets:
    print("lemmas: ", wit.lemmas())
    print("definition: ", wit.definition())
    print ('-' * 40, '\n\n')
    


[Synset('wit.n.01'), Synset('brain.n.02'), Synset('wag.n.01')]

lemmas:  [Lemma('wit.n.01.wit'), Lemma('wit.n.01.humor'), Lemma('wit.n.01.humour'), Lemma('wit.n.01.witticism'), Lemma('wit.n.01.wittiness')]
definition:  a message whose ingenuity or verbal skill or incongruity has the power to evoke laughter
---------------------------------------- 


lemmas:  [Lemma('brain.n.02.brain'), Lemma('brain.n.02.brainpower'), Lemma('brain.n.02.learning_ability'), Lemma('brain.n.02.mental_capacity'), Lemma('brain.n.02.mentality'), Lemma('brain.n.02.wit')]
definition:  mental ability
---------------------------------------- 


lemmas:  [Lemma('wag.n.01.wag'), Lemma('wag.n.01.wit'), Lemma('wag.n.01.card')]
definition:  a witty amusing person who makes jokes
---------------------------------------- 




In [96]:
# Manually select the synsets that are relevant to us
relevant_synsets = {}
relevant_synsets['Ravenclaw'] = [wn.synset('intelligence.n.01'), wn.synset('wit.n.01'), wn.synset('brain.n.02'),
                                 wn.synset('wisdom.n.01'), wn.synset('wisdom.n.02'), wn.synset('wisdom.n.03'),
                                 wn.synset('wisdom.n.04'), wn.synset('creativity.n.01'), wn.synset('originality.n.01'),
                                 wn.synset('originality.n.02'), wn.synset('individuality.n.01'), wn.synset('credence.n.01'),
                                 wn.synset('acceptance.n.03')]
relevant_synsets['Hufflepuff'] = [wn.synset('dedication.n.01'), wn.synset('commitment.n.04'), wn.synset('commitment.n.02'),
                                  wn.synset('diligence.n.01'), wn.synset('diligence.n.02'), wn.synset('application.n.06'),
                                  wn.synset('fairness.n.01'), wn.synset('fairness.n.01'), wn.synset('patience.n.01'),
                                  wn.synset('kindness.n.01'), wn.synset('forgivingness.n.01'), wn.synset('kindness.n.03'),
                                  wn.synset('tolerance.n.03'), wn.synset('tolerance.n.04'), wn.synset('doggedness.n.01'),
                                  wn.synset('loyalty.n.01'), wn.synset('loyalty.n.02')]
relevant_synsets['Gryffindor'] = [wn.synset('courage.n.01'), wn.synset('fearlessness.n.01'), wn.synset('heart.n.03'),
                                  wn.synset('boldness.n.02'), wn.synset('chivalry.n.01'), wn.synset('boldness.n.01')]
relevant_synsets['Slytherin'] = [wn.synset('resourcefulness.n.01'), wn.synset('resource.n.03'), wn.synset('craft.n.05'),
                                 wn.synset('cunning.n.02'), wn.synset('ambition.n.01'), wn.synset('ambition.n.02'),
                                 wn.synset('determination.n.02'), wn.synset('determination.n.04'),
                                 wn.synset('self-preservation.n.01'), wn.synset('brotherhood.n.02'),
                                 wn.synset('inventiveness.n.01'), wn.synset('brightness.n.02'), wn.synset('ingenuity.n.02')]

In [97]:
# Function that will get the different word forms from a lemma
def get_forms(lemma):
    drfs = lemma.derivationally_related_forms()
    output_list = []
    if drfs:
        for drf in drfs:
            drf_pos = str(drf).split(".")[1]
            if drf_pos in ['n', 's', 'a']:
                output_list.append(drf.name().lower())
                if drf_pos in ['s', 'a']:
                    # Adverbs + "-ness" nouns + comparative & superlative adjectives
                    if len(drf.name()) == 3:
                        last_letter = drf.name()[-1:]
                        output_list.append(drf.name().lower() + last_letter + 'er')
                        output_list.append(drf.name().lower() + last_letter + 'est')
                        output_list.append(drf.name().lower()+'ness')
                        output_list.append(drf.name().lower()+'ly')
                    elif drf.name()[-4:] in ['able', 'ible']:
                        output_list.append(drf.name().lower()+'r')
                        output_list.append(drf.name().lower()+'st')
                        output_list.append(drf.name().lower()+'ness')
                        output_list.append(drf.name()[:-1].lower()+'y')
                    elif drf.name()[-1:] == 'e':
                        output_list.append(drf.name().lower()+'r')
                        output_list.append(drf.name().lower()+'st')
                        output_list.append(drf.name().lower()+'ness')
                        output_list.append(drf.name().lower()+'ly')
                    elif drf.name()[-2:] == 'ic':
                        output_list.append(drf.name().lower()+'er')
                        output_list.append(drf.name().lower()+'est')
                        output_list.append(drf.name().lower()+'ness')
                        output_list.append(drf.name().lower()+'ally')
                    elif drf.name()[-1:] == 'y':
                        output_list.append(drf.name()[:-1].lower()+'ier')
                        output_list.append(drf.name()[:-1].lower()+'iest')
                        output_list.append(drf.name()[:-1].lower()+'iness')
                        output_list.append(drf.name()[:-1].lower()+'ily')
                    else:
                        output_list.append(drf.name().lower()+'er')
                        output_list.append(drf.name().lower()+'est')
                        output_list.append(drf.name().lower()+'ness')
                        output_list.append(drf.name().lower()+'ly')
        return output_list
    else:
        return output_list

In [98]:
# Creates a copy of our trait dictionary
import copy
new_trait_dict = copy.deepcopy(trait_dict)
antonym_dict = {}

In [100]:
# Add synonyms and word forms to the (new) trait dictionary; also add antonyms (and their word forms) to the antonym dictionary
for house, traits in trait_dict.items():
    antonym_dict[house] = []
    for trait in traits:
        synsets = wn.synsets(trait, pos=wn.NOUN)
        for synset in synsets:
            if synset in relevant_synsets[house]:
                for lemma in synset.lemmas():
                    new_trait_dict[house].append(lemma.name().lower())
                    if get_forms(lemma):
                        new_trait_dict[house].extend(get_forms(lemma))
                    if lemma.antonyms():
                        for ant in lemma.antonyms():
                            antonym_dict[house].append(ant.name().lower())
                            if get_forms(ant):
                                antonym_dict[house].extend(get_forms(ant))


In [105]:
# Print some of our results
print("Gryffindor traits: {}".format(list(set(new_trait_dict['Gryffindor']))))
print('-' * 40, '\n\n')

print("Gryffindor anti-traits: {}".format(list(set(antonym_dict['Gryffindor']))))
print('-' * 40, '\n\n')

Gryffindor traits: ['bravely', 'spunkier', 'braveness', 'nervily', 'spunkiness', 'nerviness', 'cheekiest', 'bravest', 'cheekily', 'cheeky', 'hardiness', 'courageously', 'spunk', 'bravery', 'politesse', 'mettle', 'daring', 'face', 'nervy', 'boldness', 'cheek', 'courageousest', 'nervier', 'fearlessly', 'brassy', 'cheekiness', 'braver', 'brassiest', 'heart', 'bolder', 'fearlessest', 'brave', 'spunkiest', 'nerve', 'spunkily', 'courageousness', 'fearlessness', 'spunky', 'brassier', 'hardihood', 'nerviest', 'brassiness', 'courageous', 'brassily', 'boldly', 'boldest', 'chivalry', 'gallantry', 'cheekier', 'bold', 'fearlesser', 'brass', 'courage', 'courageouser', 'fearless']
---------------------------------------- 


Gryffindor anti-traits: ['timidness', 'timider', 'timid', 'timidest', 'cowardice', 'fear', 'timidity', 'timidly']
---------------------------------------- 




In [113]:
from itertools import combinations
print(list(combinations(list(new_trait_dict.keys()), 2)))

[('Gryffindor', 'Slytherin'), ('Gryffindor', 'Ravenclaw'), ('Gryffindor', 'Hufflepuff'), ('Slytherin', 'Ravenclaw'), ('Slytherin', 'Hufflepuff'), ('Ravenclaw', 'Hufflepuff')]


In [118]:
'''Test Overlap Words Among Houses'''
# Test that the trait dictionary and the antonym dictionary don't have any repeats among clusters
from itertools import combinations
def test_overlap(dict):
    results = []
    house_combos = combinations(list(dict.keys()), 2)
    for combo in house_combos:
        results.append(set(dict[combo[0]]).isdisjoint(dict[combo[1]]))
    return results

# Outputs results from our test; should output "False"
print("Any words overlap in trait dictionary? {}".format(sum(test_overlap(new_trait_dict)) !=6 ))
print("Any words overlap in antonym dictionary? {}".format(sum(test_overlap(antonym_dict)) !=6))

Any words overlap in trait dictionary? False
Any words overlap in antonym dictionary? False


In [166]:
import operator
dict={'Gryffindor': 1, 'Hufflepuff': 0, 'Ravenclaw': 0, 'Slytherin': 0}
max(dict.items(), key=operator.itemgetter(1))[0]

'Gryffindor'

In [167]:
max(dict.items(), key=operator.itemgetter(1))[1]

1

In [170]:
'''Scan through the "text" column in our DataFrame and count the number of times a characteristic appears'''
'''Build The Function'''
# Imports "word_tokenize", which breaks up sentences into words and punctuation
from nltk import word_tokenize
def sort_student(text):
    text_list = word_tokenize(text)
    text_list = [word.lower() for word in text_list]
    score_dict = {}
    houses = ['Gryffindor', 'Hufflepuff', 'Ravenclaw', 'Slytherin']
    for house in houses:
        score_dict[house] = (sum([True for word in text_list if word in new_trait_dict[house]]) -
                                  sum([True for word in text_list if word in antonym_dict[house]]))
    sorted_house = max(dict.items(), key=operator.itemgetter(1))[0]
    sorted_score = max(dict.items(), key=operator.itemgetter(1))[1]
    if sum([True for i in score_dict.values() if i ==sorted_score])==1:
        return sorted_house
    else:
        return 'Tie'

In [172]:
'''Test Function'''
# Test our function
print(sort_student('Alice was brave'))
print(sort_student('Alice was British'))

Gryffindor
Tie


In [173]:
import warnings
warnings.filterwarnings("ignore")

In [181]:
mydf_relevant['new_house'] = mydf_relevant['text'].map(sort_student)
mydf_relevant.head(20)
mydf_relevant.sort_index().head(20)

,id,title,url,house,text,text_len,new_house
15,325,Katie Bell,/wiki/Katie_Bell,Gryffindor,Katie was a friendly person who was very inter...,698,Tie
22,31,Sirius Black,/wiki/Sirius_Black,Gryffindor,Sirius was true to the ideal of a Gryffindor s...,3483,Gryffindor
29,20,Lavender Brown,/wiki/Lavender_Brown,Gryffindor,Lavender was a somewhat silly and sentimental ...,585,Gryffindor
44,449,Colin Creevey,/wiki/Colin_Creevey,Gryffindor,Colin was a person who was very easily excited...,1552,Gryffindor
45,451,Dennis Creevey,/wiki/Dennis_Creevey,Gryffindor,Dennis apparently had a similar personality to...,185,Gryffindor
53,4178,Albus Dumbledore,/wiki/Albus_Dumbledore,Gryffindor,Considered to be the most powerful wizard of h...,7789,Tie
54,27337,Fay Dunbar,/wiki/Fay_Dunbar,Gryffindor,"Fay was more outgoing than her friend, openly ...",413,Tie
55,30812,Fay Dunbar's friend,/wiki/Fay_Dunbar%27s_friend,Gryffindor,The girl appeared to be very different academi...,627,Gryffindor
62,646,Fat Lady,/wiki/Fat_Lady,Gryffindor,She was well known for her indulgence in good ...,152,Tie
63,21,Seamus Finnigan,/wiki/Seamus_Finnigan,Gryffindor,"Through his school years, Seamus generally app...",1467,Gryffindor


In [182]:
print("Match rate: {}".format(sum(mydf_relevant['house'] == mydf_relevant['new_house']) / len(mydf_relevant)))
print("Percentage of ties: {}".format(sum(mydf_relevant['new_house'] == 'Tie') / len(mydf_relevant)))

Match rate: 0.16666666666666666
Percentage of ties: 0.5625
